<a href="https://colab.research.google.com/github/kridtapon/Moving-average-crossover-multiple-timeframe/blob/main/Moving_average_crossover_multiple_timeframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

In [ ]:
pip install --upgrade yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate moving average crossover
def calculate_sma(df, short_window=50, long_window=200):
    short_sma = df['Close'].rolling(window=short_window).mean()
    long_sma = df['Close'].rolling(window=long_window).mean()
    return short_sma, long_sma

# Define the stock symbol and time period
symbol = 'NFLX'
start_date = '2019-01-01'
end_date = '2025-03-31'

# Download the stock data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate SMA for weekly and daily data
weekly_data = df.resample('W').last()  # Resample weekly data
weekly_data['Short_SMA'], weekly_data['Long_SMA'] = calculate_sma(weekly_data, short_window=7, long_window=28)
daily_data = df.copy()  # Use the original daily data
daily_data['Short_SMA'], daily_data['Long_SMA'] = calculate_sma(daily_data, short_window=1, long_window=136)

# Forward-fill weekly SMA to daily data
weekly_data = weekly_data.reindex(daily_data.index, method='ffill')
daily_data['HTF_Trend'] = weekly_data['Short_SMA'] > weekly_data['Long_SMA']  # Weekly trend confirmation

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2025) & (df.index.year <= 2025)]
daily_data = daily_data[(daily_data.index.year >= 2025) & (daily_data.index.year <= 2025)]

# Generate entry and exit signals for the strategy
entries = (daily_data['HTF_Trend']) & (daily_data['Short_SMA'] > daily_data['Long_SMA'])
exits = (~daily_data['HTF_Trend']) & (daily_data['Short_SMA'] < daily_data['Long_SMA'])

# Backtest on training data
portfolio = vbt.Portfolio.from_signals(
            close=df['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()